In [17]:
import sympy as sp

def drazin_inverse(A, tol=1e-10):
    A = sp.Matrix(A)
    n = A.shape[0]

    def rank_k(A, k):
        """ Compute rank of A^k """
        return (A**k).rank()

    # Find the Drazin index k
    k = 0
    while rank_k(A, k) != rank_k(A, k + 1):
        k += 1

    # Compute the range space of A^k
    Ak = A**k
    r = Ak.rank()

    # If A is full-rank, return its inverse directly
    if r == n:
        return A.inv()

    # Compute null space of A^k
    nullspace_basis = Ak.nullspace()
    
    # Construct Q from the null space basis
    if nullspace_basis:
        Q = sp.Matrix.hstack(*nullspace_basis)
    else:
        raise ValueError("Null space computation failed.")

    # Ensure Q is a square matrix
    if Q.shape[1] < n:
        # Add missing columns from the identity matrix
        extra_cols = sp.eye(n).extract(range(n), range(Q.shape[1], n))
        P = sp.Matrix.hstack(Q, extra_cols)
    else:
        P = Q

    # Avoid inverting P, solve in transformed basis
    A_reordered = P.T * A * P

    # Extract M (invertible part) and N (nilpotent part)
    M = A_reordered[:r, :r]
    N = A_reordered[r:, r:]

    # Compute M inverse (only if M is invertible)
    M_inv = M.inv() if M.det() != 0 else sp.zeros(r, r)

    # Construct the Drazin inverse
    AD = P * sp.Matrix.vstack(
        sp.Matrix.hstack(M_inv, sp.zeros(r, n - r)),
        sp.Matrix.hstack(sp.zeros(n - r, r), sp.zeros(n - r, n - r))
    ) * P.T

    return AD


In [25]:
import sympy as sp

# Define symbols
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p = sp.symbols('a b c d e f g h i j k l m n o p')
i = sp.I  # Imaginary unit

# Example 4x4 matrix (singular, to demonstrate Drazin inverse)
A = sp.Matrix([[a, b],
               [e, f]
])

# Compute Drazin inverse using your function
AD = drazin_inverse(A)

In [26]:
AD = AD.applyfunc(lambda expr: sp.simplify(sp.expand(expr)))

In [27]:
AD

Matrix([
[ f/(a*f - b*e), -b/(a*f - b*e)],
[-e/(a*f - b*e),  a/(a*f - b*e)]])

In [ ]:
import sympy as sp

# Define variables
chi, A, B = sp.symbols('chi A B', real=True)
i = sp.I  # Imaginary unit

# Define the eigenvalue function
lambda_chi = (B * (1 + sp.exp(i * chi)) + sp.sqrt(B**2 * (1 + sp.exp(i * chi))**2 - 4 * (A**2 * sp.exp(-i * chi) + B**2))) / 2

# Compute the real part
real_part = sp.re(lambda_chi)

# Find the value of chi that minimizes the real part
chi_min = sp.solve(sp.diff(real_part, chi), chi)

# Print the result
print("Critical points for minimum real part of lambda:", chi_min)

# Evaluate the real part at these critical points
real_part_min = [real_part.subs(chi, c) for c in chi_min]

print("Minimum real part values:", real_part_min)
